In [22]:
# calculate dM/M0 time series for every ROI
import os
import sys
import pandas as pd
import subprocess
import glob
import re
import numpy as np

local_dir = '../data_refine'
roi_dir = '../rois'


In [23]:
def refine_list(lst):
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra \n\n
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [24]:
cols = ['subject_id', 'model', 'scalar','GM','WM','acc','caudate', 'thalamus', 'S1', ]

df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    # skip the uneven sessions for now
    if '4907' in root or '5509' in root or '0000' in root or '7586' in root:
        continue
    if 'sub-' in root and 'hex' not in root and 'baseline' not in root:
        scalar_maps = glob.glob(root+"/**/*acq-glymph*.nii.gz", recursive=True)
        sub_name = root.split('/')[4].split('sub-')[1]
        model = root.split('/')[2]    
        scalar = root.split('/')[3]
            
        if len(scalar_maps) > 0:
            # merge scalar volumes into one time series volume
            scalar_ts = os.path.join(root, 'sub-{}_{}_{}_ts.nii.gz'.format(sub_name, model, scalar))
            scalar_maps.sort()
            print(scalar_ts)
            pattern = "x|(.*?)|_space"
            for s in scalar_maps:
                substring = s.split('full')[1].split('_space')[0]
                print(substring)
            print('')
            
            fslmerge_inputs = ' '.join(scalar_maps)
            cmd1="fslmerge -t {} {}".format(scalar_ts, fslmerge_inputs)
            print(cmd1)
            subprocess.check_output(cmd1, shell=True)
            
            for mask in glob.glob(roi_dir+"/*mask.nii.gz"):
                cmd2="fslmeants --transpose -i '{}' -m '{}'".format(scalar_ts, mask)
                result = refine_list(subprocess.check_output(cmd2, shell=True))
                if 'gm' in mask:
                    gm = result
                elif 'wm' in mask:
                    wm = result
                if 'acc' in mask:
                    acc = result
                elif 'caudate' in mask:
                    caudate = result
                elif 'thal' in mask:
                    thalamus = result
                elif 's1' in mask:
                    s1 = result


            df.loc[len(df.index)] = [sub_name, model, scalar, gm, wm, acc, caudate, thalamus, s1]

df.to_csv('ts_data.csv', index=False)
df.head()

../data_refine/GQI/iso/sub-MB5330/sub-MB5330_GQI_iso_ts.nii.gz
1501
1504
1514
1517
1527
1530
1553
1556
1606
1609
1619
1621

fslmerge -t ../data_refine/GQI/iso/sub-MB5330/sub-MB5330_GQI_iso_ts.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1501_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1504_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1514_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1517_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1527_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ../data_refine/GQI/iso/sub-MB5330/baseline/sub-MB5330_se

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,S1
0,MB5330,GQI,iso,"[0.200636, 0.207349, 0.192243, 0.205165, 0.216...","[0.207202, 0.215115, 0.19871, 0.212884, 0.2242...","[0.208031, 0.219377, 0.20253, 0.218277, 0.2291...","[0.209803, 0.221403, 0.204345, 0.220674, 0.235...","[0.185593, 0.197859, 0.18052, 0.195454, 0.2081...","[0.178121, 0.185507, 0.171495, 0.182166, 0.196..."
1,AG6470,GQI,iso,"[0.244378, 0.236701, 0.251958, 0.24621, 0.2342...","[0.252908, 0.246563, 0.261418, 0.255285, 0.243...","[0.274828, 0.270195, 0.286724, 0.279462, 0.266...","[0.302385, 0.297936, 0.307736, 0.303713, 0.286...","[0.259048, 0.256311, 0.265161, 0.26215, 0.2482...","[0.224549, 0.21504, 0.233589, 0.229021, 0.2164..."
2,AM5508,GQI,iso,"[0.207368, 0.197094, 0.186958, 0.189082, 0.186...","[0.201513, 0.192101, 0.183694, 0.183691, 0.181...","[0.232884, 0.219202, 0.212976, 0.213258, 0.211...","[0.264339, 0.253393, 0.245846, 0.241851, 0.243...","[0.200452, 0.190934, 0.181494, 0.178636, 0.178...","[0.183249, 0.172481, 0.164691, 0.167208, 0.166..."
3,AA5325,GQI,iso,"[0.21421, 0.197221, 0.220012, 0.203542, 0.2038...","[0.22085, 0.20253, 0.226644, 0.209422, 0.20945...","[0.24506, 0.22495, 0.250746, 0.230565, 0.22943...","[0.270753, 0.251591, 0.283037, 0.260538, 0.259...","[0.218984, 0.199904, 0.224873, 0.204398, 0.205...","[0.184795, 0.169679, 0.194407, 0.177466, 0.180..."
4,KS5471,GQI,iso,"[0.18363, 0.186319, 0.196978, 0.184978, 0.1902...","[0.186871, 0.191168, 0.20084, 0.18917, 0.19432...","[0.212126, 0.215157, 0.22886, 0.217496, 0.2204...","[0.213795, 0.217234, 0.225773, 0.212375, 0.213...","[0.17819, 0.186217, 0.189658, 0.180295, 0.1813...","[0.158769, 0.159915, 0.169831, 0.159233, 0.163..."


### Normalize time series data to baseline

In [25]:
def normalize_ts(ts, fullbaseline):
    # normalize ts based on the first baseline timepoint
    normalized_ts = []
    
    if fullbaseline:
        baseline = sum(ts[0:6])/6
    else:
        baseline = ts[0]
    
    for tpoint in ts:
        norm_tpoint = 100*(tpoint - baseline)/baseline
        normalized_ts.append(norm_tpoint)
        
    return normalized_ts

In [26]:
# TESTING THE FUNCTION
ts = [0.200636, 0.207349, 0.192243, 0.205165, 0.216784, 0.202897, 0.219422, 0.219491, 0.222456, 0.221574, 0.215319, 0.227749]
new_ts = normalize_ts(ts, True)
print(new_ts)

0.20417900000000003
[-1.7352421159864717, 1.552559273970378, -5.845850944514386, 0.4829096038279936, 6.1735046209453355, -0.6278804382429305, 7.46550820603489, 7.499302082976194, 8.951459258787612, 8.519485353537808, 5.455996943858076, 11.543792456618936]


In [29]:
norm_df = df.copy()
for row, row in df.iterrows(): #iterate over rows
    row_list = [row.subject_id, row.model, row.scalar] 
    sub_id = row.subject_id
    for c, value in row.items():
        if type(value) == list:
            norm_ts = normalize_ts(value, False) 
            row_list.append(norm_ts)
            
    norm_df.loc[len(norm_df.index)] = row_list

norm_df = norm_df.tail(40) # only get the newly created rows
norm_df.reset_index(drop=True)
norm_df.to_csv('ts_data_norm_t0.csv', index=False)


0.200636
0.207202
0.208031
0.209803
0.185593
0.178121
0.244378
0.252908
0.274828
0.302385
0.259048
0.224549
0.207368
0.201513
0.232884
0.264339
0.200452
0.183249
0.21421
0.22085
0.24506
0.270753
0.218984
0.184795
0.18363
0.186871
0.212126
0.213795
0.17819
0.158769
0.241493
0.247928
0.268242
0.287913
0.238921
0.223528
0.219057
0.220062
0.244099
0.256664
0.205654
0.18675
0.147259
0.153068
0.173379
0.180433
0.149932
0.134934
0.203094
0.202976
0.228547
0.260413
0.192071
0.181802
0.778345
0.668629
0.817634
0.831412
0.788978
0.825499
0.752914
0.657484
0.771895
0.742391
0.746423
0.812346
0.742525
0.648783
0.801825
0.687986
0.701747
0.839683
0.776342
0.668457
0.781589
0.804213
0.729171
0.867733
0.788418
0.672802
0.821535
0.854624
0.778073
0.893704
0.770766
0.668813
0.797538
0.752157
0.718197
0.825643
0.755686
0.656312
0.786338
0.802987
0.749716
0.820386
0.797641
0.671218
0.799934
0.781939
0.750232
0.870552
0.75237
0.660591
0.7879
0.737731
0.727622
0.840689
0.029018
0.063249
0.032122
0.04113
0.

In [28]:
norm_df

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,S1
40,MB5330,GQI,iso,"[-1.7352421159864717, 1.552559273970378, -5.84...","[-1.9892088259316292, 1.7538022963567819, -6.0...","[-3.4082968266460387, 1.8598096776964563, -5.9...","[-3.8042917589089518, 1.5143653413072329, -6.3...","[-4.069118326108385, 2.271035637736988, -6.691...","[-2.1747667016023082, 1.8816734325871796, -5.8..."
41,AG6470,GQI,iso,"[0.2817772209105949, -2.86851946588172, 3.3922...","[-0.0005271981526835675, -2.5093314073027417, ...","[-0.8126423327105629, -2.4847246098895686, 3.4...","[0.3679899627031338, -1.1087274252098576, 2.14...","[-0.2966810848328284, -1.3501074145895202, 2.0...","[-0.20059229541882073, -4.4268082565803555, 3...."
42,AM5508,GQI,iso,"[7.803438909322492, 2.4623422533563835, -2.807...","[7.514981321699825, 2.493315204874413, -1.9921...","[7.49732661460939, 1.181828672539151, -1.69204...","[6.093701607492505, 1.700472996520933, -1.3285...","[8.185086672237624, 3.048167834080071, -2.0466...","[7.9162577048408, 1.5749283498881086, -3.01264..."
43,AA5325,GQI,iso,"[2.2285287955699937, -5.87921816165861, 4.9974...","[2.4831534277447838, -6.01805268860697, 5.1717...","[2.945336107742678, -5.502516292186744, 5.3339...","[1.2302659251297021, -5.9341029116231185, 5.82...","[3.2772891267951096, -5.72122526941307, 6.0546...","[0.9262748567714751, -7.329368265255402, 6.175..."
44,KS5471,GQI,iso,"[-2.672276642600789, -1.2470506549732285, 4.40...","[-3.0210508288688103, -0.7910710856858111, 4.2...","[-3.1449307235539834, -1.7609998759591308, 4.4...","[-1.3251569042082187, 0.26207752787123906, 4.2...","[-2.452420989052206, 1.941845898656861, 3.8255...","[-2.525938811009924, -1.822367747876797, 4.265..."
45,PER5474,GQI,iso,"[5.37975995409412, 0.7101885326915836, 1.12953...","[4.964765203671726, 0.9038199783094387, 1.1167...","[4.043027713312345, 0.4881990548900786, 2.3495...","[3.362077631144347, 0.05456869193100847, 2.322...","[3.505774845989657, 0.010541729363476226, 2.31...","[7.411420704501829, 2.66861120254487, -1.11842..."
46,CM5268,GQI,iso,"[0.7764824051249345, -3.113673742442756, 1.451...","[1.1051043692665734, -3.040874772194751, 0.837...","[0.13379943212157888, -2.994933124213324, -0.1...","[0.1841716471945231, -4.14890385016976, 0.3574...","[0.3656985199621378, -3.7303526518022982, 1.00...","[-0.5613116499900135, -4.065493754575909, 1.70..."
47,CKA6605,GQI,iso,"[-0.6190822196077144, 0.147571924441379, -5.28...","[-0.1347256066729941, -0.33762954698922787, -5...","[0.6487230170409647, 0.15296511408528757, -5.0...","[-0.7648435938849245, -0.4090043283694447, -6....","[-0.3497096092047512, -0.4753259211585872, -6....","[0.47606755294325204, -1.069786836905931, -6.0..."
48,EP5270,GQI,iso,"[2.306626703271793, -1.2422235095584697, -1.56...","[2.173584753697303, -0.7510394748764174, -1.44...","[1.2825749288915191, -0.5006953897469205, -0.8...","[1.2394474613324629, -0.6942694102241241, -1.0...","[1.684318080147375, 0.13897013862600677, -1.67...","[3.6297566513554407, -1.221065831912256, -1.55..."
49,MB5330,GQI,md,"[0.2510962518458761, 0.265521889294831, 0.4111...","[0.11254561715924472, 0.04846199302068275, 0.1...","[0.8700336589568389, 0.6372378682797771, 0.030...","[2.6543347700425177, 1.2992529043715584, 0.310...","[1.1056793117787547, -0.8556205240427308, -0.0...","[-0.16204286486302164, 1.1461927384881312, 0.6..."
